Import dữ liệu từ group Chia sẻ kiến thức công nghệ UEH

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


In [22]:
df = pd.read_json('dataset_facebook-groups-scraper_2025-05-09_02-32-18-790.json')
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   facebookUrl         167 non-null    object 
 1   url                 167 non-null    object 
 2   time                167 non-null    object 
 3   user                167 non-null    object 
 4   text                167 non-null    object 
 5   topReactionsCount   167 non-null    int64  
 6   feedbackId          167 non-null    object 
 7   reactionHahaCount   38 non-null     float64
 8   reactionSadCount    20 non-null     float64
 9   id                  167 non-null    object 
 10  legacyId            167 non-null    int64  
 11  attachments         80 non-null     object 
 12  likesCount          167 non-null    int64  
 13  sharesCount         167 non-null    int64  
 14  commentsCount       167 non-null    int64  
 15  topComments         167 non-null    object 
 16  facebook

# Mô tả các trường dữ liệu bài viết trong Group Facebook

| **Tên trường**          | **Mô tả**                                                                                     |
|-------------------------|---------------------------------------------------------------------------------------------|
| `facebookUrl`          | URL của group Facebook mà bài viết được đăng lên, giúp truy cập trực tiếp vào group.           |
| `url`                  | URL trực tiếp đến bài viết cụ thể trong group.                                                 |
| `time`                 | Thời gian bài viết được đăng tải, bao gồm ngày và giờ theo định dạng ISO 8601.                 |
| `user`                 | Thông tin của người đăng bài, bao gồm `id` và `name`.                                          |
| `text`                 | Nội dung văn bản của bài viết, bao gồm các bình luận hoặc chia sẻ thông tin của người dùng.     |
| `topReactionsCount`    | Tổng số lượt tương tác (Reaction) của bài viết, bao gồm Like, Love, Haha, Wow, Sad, Angry.     |
| `feedbackId`           | ID phản hồi của Facebook, dùng để theo dõi hoặc cập nhật trạng thái tương tác.                |
| `reactionHahaCount`    | Số lượng lượt "Haha" (cười vui) mà bài viết nhận được.                                         |
| `reactionSadCount`     | Số lượng lượt "Sad" (buồn) mà bài viết nhận được.                                              |
| `id`                   | ID duy nhất của bài viết trên Facebook, dùng để phân biệt với các bài viết khác.               |
| `legacyId`             | ID dạng cũ của bài viết trên Facebook, trước khi chuyển sang dạng UUID.                         |
| `attachments`          | Danh sách các tập tin đính kèm, có thể là ảnh, video hoặc tài liệu. Mỗi tập tin gồm các thông tin sau: |
| - `thumbnail`          | URL của ảnh đại diện thu nhỏ cho ảnh hoặc video.                                               |
| - `__typename`         | Kiểu dữ liệu của đính kèm (`Photo`, `Video`, v.v.).                                           |
| - `photo_image`        | Thông tin chi tiết về ảnh bao gồm:                                                             |
|   - `uri`              | Đường dẫn đến ảnh đầy đủ.                                                                     |
|   - `height`           | Chiều cao của ảnh (pixel).                                                                    |
|   - `width`            | Chiều rộng của ảnh (pixel).                                                                   |
| - `__isMedia`          | Xác định đây là loại phương tiện (`Photo`, `Video`).                                          |
| - `accent_color`       | Mã màu nổi bật của ảnh.                                                                       |
| - `photo_product_tags` | Thẻ sản phẩm liên kết với ảnh (nếu có).                                                       |
| - `url`                | URL trực tiếp đến ảnh hoặc video trong bài viết.                                               |
| - `id`                 | ID của ảnh hoặc video trên Facebook.                                                           |
| - `ocrText`            | Văn bản nhận diện từ ảnh (nếu có OCR - Optical Character Recognition).                         |


### Lý do loại bỏ các trường trong quá trình lọc dữ liệu

Trong quá trình xử lý và phân tích dữ liệu bài viết từ Group Facebook, các trường sau đây sẽ được loại bỏ:

| **Tên trường**           | **Lý do loại bỏ**                                                                                      |
|------------------------- |------------------------------------------------------------------------------------------------------|
| `facebookUrl`           | URL của Group Facebook không cần thiết vì đã có URL trực tiếp đến bài viết (`url`).                     |
| `topReactionsCount`     | Tổng số lượt tương tác (Reaction) không cần thiết khi đã có thông tin chi tiết của các loại reaction khác. |
| `feedbackId`            | ID phản hồi của Facebook, chủ yếu dùng để tracking nội bộ, không cần thiết cho phân tích nội dung.       |
| `reactionHahaCount`     | Lượt "Haha" không cần thiết vì không phục vụ cho mục tiêu phân tích nội dung chính.                    |
| `reactionSadCount`      | Lượt "Sad" cũng tương tự, không cung cấp thông tin phân tích hữu ích cho bài viết.                     |
| `likesCount`            | Thống kê lượt "Like" chỉ là một dạng tương tác chung, không mang lại thông tin chi tiết như nội dung hoặc ảnh. |
| `sharesCount`           | Lượt chia sẻ bài viết không phục vụ cho phân tích nội dung bài viết gốc mà chỉ là hành vi lan truyền.   |
| `commentsCount`         | Số lượng bình luận không cần thiết khi ta có thể phân tích trực tiếp nội dung bình luận sau này.       |
| `topComments`           | Bình luận nổi bật có thể gây nhiễu khi phân tích bài viết gốc, vì thường là những tương tác ngẫu nhiên.  |
| `facebookId`            | ID của Facebook người dùng không cần thiết cho phân tích nội dung.                                    |
| `groupTitle`            | Tiêu đề của group không quan trọng khi phân tích nội dung bài viết.                                    |
| `inputUrl`              | URL đầu vào, không còn cần thiết sau khi đã trích xuất thông tin bài viết.                             |
| `reactionLoveCount`     | Lượt "Love" (yêu thích) không cần thiết cho mục tiêu phân tích.                                        |
| `reactionLikeCount`     | Lượt "Like" cũng tương tự, chỉ là một dạng phản ứng bề mặt.                                             |
| `reactionCareCount`     | Lượt "Care" ít giá trị phân tích so với nội dung bài viết.                                             |
| `reactionWowCount`      | Lượt "Wow" không cung cấp thêm thông tin hữu ích.                                                      |
| `previewTitle`          | Tiêu đề xem trước của link đính kèm không cần thiết cho phân tích nội dung chính của bài viết.          |
| `previewDescription`    | Mô tả xem trước của link cũng không cần thiết khi nội dung chính đã đủ thông tin.                      |
| `previewTarget`         | Đích đến của link không quan trọng khi phân tích nội dung văn bản của bài viết.                         |
| `previewSource`         |Không quan trọng|



In [23]:
fields_to_drop = [
    'facebookUrl',
    'topReactionsCount',
    'feedbackId',
    'reactionHahaCount',
    'reactionSadCount',
    'likesCount',
    'sharesCount',
    'commentsCount',
    'topComments',
    'facebookId',
    'groupTitle',
    'inputUrl',
    'reactionLoveCount',
    'reactionLikeCount',
    'reactionCareCount',
    'reactionWowCount',
    'previewTitle',
    'previewDescription',
    'previewTarget',
    'previewSource'
]

df = df.drop(columns=fields_to_drop)

df

,url,time,user,text,id,legacyId,attachments,isVideo,textReferences,link,title,sharedPost
0,https://www.facebook.com/groups/chiasekienthuc...,2025-03-20T04:31:16.000Z,"{'id': '457965046892506', 'name': 'Anonymous p...",Dốc ọp the day 👇,UzpfSTQ1Nzk2NTA0Njg5MjUwNjpWSzoxNzM4NzAxMDUzNj...,1738701053691877,[{'thumbnail': 'https://scontent.fbel16-1.fna....,NaN,NaN,NaN,NaN,NaN
1,https://www.facebook.com/groups/chiasekienthuc...,2025-03-12T13:00:09.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Excel có gì] Official #02 - Conditional For...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwNDM1MDgxMT...,1730435081185141,[{'thumbnail': 'https://scontent.fbel16-1.fna....,1.0,NaN,NaN,NaN,NaN
2,https://www.facebook.com/groups/chiasekienthuc...,2025-03-09T13:00:06.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwMjMyNzk3OD...,1730232797872036,"[{'mediaset_token': 'pcb.1730232797872036', 'u...",NaN,NaN,NaN,NaN,NaN
3,https://www.facebook.com/groups/chiasekienthuc...,2025-02-27T11:39:11.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzIyMTc5Mzc1Mz...,1722179375344045,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN
4,https://www.facebook.com/groups/chiasekienthuc...,2025-01-28T17:05:04.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,Chúc các thành viên nhóm Chia sẻ kiến thức côn...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzAxNjc5MjY0MD...,1701679264060723,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
162,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:43:22.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTk3OTg2MD...,956199798608677,[{'thumbnail': 'https://scontent-ord5-1.xx.fbc...,NaN,NaN,NaN,NaN,NaN
163,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",Nhóm được thành lập bởi Câu lạc bộ Công nghệ K...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4Njg2MD...,956192868609370,NaN,NaN,NaN,NaN,NaN,NaN
164,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4NjE5ND...,956192861942704,NaN,NaN,NaN,NaN,NaN,NaN
165,https://www.facebook.com/groups/chiasekienthuc...,2021-10-22T09:55:19.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTU2NDAyMjg2Nj...,955640228664634,[{'thumbnail': 'https://scontent.fjed5-1.fna.f...,NaN,NaN,NaN,NaN,NaN


Trích hashtag và title từ bài
Duyệt các bài không trích được title và hasgtag để xử lý

In [24]:
df['hashtags'] = df['text'].str.findall(r'#(\w+)\b')
df['titles'] = df['text'].str.findall(r'\*\*(.*?)\*\*')
df

# Điều kiện lọc: titles là [] và hashtags là []
filtered_df = df[(df['titles'].apply(len) == 0) & (df['hashtags'].apply(len) == 0)]
filtered_df

,url,time,user,text,id,legacyId,attachments,isVideo,textReferences,link,title,sharedPost,hashtags,titles
0,https://www.facebook.com/groups/chiasekienthuc...,2025-03-20T04:31:16.000Z,"{'id': '457965046892506', 'name': 'Anonymous p...",Dốc ọp the day 👇,UzpfSTQ1Nzk2NTA0Njg5MjUwNjpWSzoxNzM4NzAxMDUzNj...,1738701053691877,[{'thumbnail': 'https://scontent.fbel16-1.fna....,NaN,NaN,NaN,NaN,NaN,[],[]
4,https://www.facebook.com/groups/chiasekienthuc...,2025-01-28T17:05:04.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,Chúc các thành viên nhóm Chia sẻ kiến thức côn...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzAxNjc5MjY0MD...,1701679264060723,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN,[],[]
8,https://www.facebook.com/groups/chiasekienthuc...,2025-01-21T16:45:59.000Z,"{'id': '1501175133909926', 'name': 'Anonymous ...",Dạ Anh Chị chỉ cho em cách sài ứng dụng văn ph...,UzpfSTE1MDExNzUxMzM5MDk5MjY6Vks6MTY4ODM3NTg3NT...,1688375875391062,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
13,https://www.facebook.com/groups/chiasekienthuc...,2024-11-21T16:49:06.000Z,{'id': 'pfbid02cFXwnKi175zKT7i1gV7Dn2wta8rCZYb...,Ố LÀ LA!!! Chào các bạn UEH-ers và toàn thể cá...,UzpfSTEwMDAyODI5MTI2MjE5NzpWSzoxNjU1MTkyMTc1Mz...,1655192175376099,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
16,https://www.facebook.com/groups/chiasekienthuc...,2024-11-10T15:24:34.000Z,"{'id': '450325650990936', 'name': 'Anonymous p...",,UzpfSTQ1MDMyNTY1MDk5MDkzNjpWSzoxNjQ3NTExNjI5ND...,1647511629477487,[{'thumbnail': 'https://scontent-cdg4-2.xx.fbc...,NaN,NaN,NaN,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:43:22.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTk3OTg2MD...,956199798608677,[{'thumbnail': 'https://scontent-ord5-1.xx.fbc...,NaN,NaN,NaN,NaN,NaN,[],[]
163,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",Nhóm được thành lập bởi Câu lạc bộ Công nghệ K...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4Njg2MD...,956192868609370,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
164,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4NjE5ND...,956192861942704,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
165,https://www.facebook.com/groups/chiasekienthuc...,2021-10-22T09:55:19.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTU2NDAyMjg2Nj...,955640228664634,[{'thumbnail': 'https://scontent.fjed5-1.fna.f...,NaN,NaN,NaN,NaN,NaN,[],[]


Trích các bài là share post

Chủ yếu các bài này bài được share từ fanpage ET nên có thể xóa bởi sẽ bị trùng

In [27]:
shared_posts = df[df['sharedPost'].notnull()]
shared_posts
df_not_shared_posts = df[df['sharedPost'].isnull()]
df_not_shared_posts

,url,time,user,text,id,legacyId,attachments,isVideo,textReferences,link,title,sharedPost,hashtags,titles
0,https://www.facebook.com/groups/chiasekienthuc...,2025-03-20T04:31:16.000Z,"{'id': '457965046892506', 'name': 'Anonymous p...",Dốc ọp the day 👇,UzpfSTQ1Nzk2NTA0Njg5MjUwNjpWSzoxNzM4NzAxMDUzNj...,1738701053691877,[{'thumbnail': 'https://scontent.fbel16-1.fna....,NaN,NaN,NaN,NaN,NaN,[],[]
1,https://www.facebook.com/groups/chiasekienthuc...,2025-03-12T13:00:09.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Excel có gì] Official #02 - Conditional For...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwNDM1MDgxMT...,1730435081185141,[{'thumbnail': 'https://scontent.fbel16-1.fna....,1.0,NaN,NaN,NaN,NaN,"[02, Tech_Learning, DA, Excelcogi]",[[Excel có gì] Official #02 - Conditional Form...
2,https://www.facebook.com/groups/chiasekienthuc...,2025-03-09T13:00:06.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwMjMyNzk3OD...,1730232797872036,"[{'mediaset_token': 'pcb.1730232797872036', 'u...",NaN,NaN,NaN,NaN,NaN,"[Tech_Learning, DA]","[[Biểu đồ đã lừa bạn như thế nào?], Cherry Pic..."
3,https://www.facebook.com/groups/chiasekienthuc...,2025-02-27T11:39:11.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzIyMTc5Mzc1Mz...,1722179375344045,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN,"[Tech_Learning, DA]","[[Biểu đồ đã lừa bạn như thế nào?], Cherry Pic..."
4,https://www.facebook.com/groups/chiasekienthuc...,2025-01-28T17:05:04.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,Chúc các thành viên nhóm Chia sẻ kiến thức côn...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzAxNjc5MjY0MD...,1701679264060723,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:43:22.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTk3OTg2MD...,956199798608677,[{'thumbnail': 'https://scontent-ord5-1.xx.fbc...,NaN,NaN,NaN,NaN,NaN,[],[]
163,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",Nhóm được thành lập bởi Câu lạc bộ Công nghệ K...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4Njg2MD...,956192868609370,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
164,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:26:25.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYxOTI4NjE5ND...,956192861942704,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
165,https://www.facebook.com/groups/chiasekienthuc...,2021-10-22T09:55:19.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTU2NDAyMjg2Nj...,955640228664634,[{'thumbnail': 'https://scontent.fjed5-1.fna.f...,NaN,NaN,NaN,NaN,NaN,[],[]


Lọc ra những bài không phải là share post có Nội dung (text) và có hashtag
Sau đó có thể thấy nên giữ lại những bài có Text, hashtag (bỏ các bài có hashtag là techmeme) 
nhưng lưu ý với các bài share post có nội dung đôi khi sẽ không phải của ET 

In [33]:
filtered_df = df_not_shared_posts[(df_not_shared_posts['text'].notnull()) & (df_not_shared_posts['text'] != '') ]
#Không nhận những bài có hashtag techMeme
filtered_df = filtered_df[~filtered_df['hashtags'].apply(lambda x: any(hashtag.lower() in ['techmeme'] for hashtag in x))]
filtered_df



,url,time,user,text,id,legacyId,attachments,isVideo,textReferences,link,title,sharedPost,hashtags,titles
0,https://www.facebook.com/groups/chiasekienthuc...,2025-03-20T04:31:16.000Z,"{'id': '457965046892506', 'name': 'Anonymous p...",Dốc ọp the day 👇,UzpfSTQ1Nzk2NTA0Njg5MjUwNjpWSzoxNzM4NzAxMDUzNj...,1738701053691877,[{'thumbnail': 'https://scontent.fbel16-1.fna....,NaN,NaN,NaN,NaN,NaN,[],[]
1,https://www.facebook.com/groups/chiasekienthuc...,2025-03-12T13:00:09.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Excel có gì] Official #02 - Conditional For...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwNDM1MDgxMT...,1730435081185141,[{'thumbnail': 'https://scontent.fbel16-1.fna....,1.0,NaN,NaN,NaN,NaN,"[02, Tech_Learning, DA, Excelcogi]",[[Excel có gì] Official #02 - Conditional Form...
2,https://www.facebook.com/groups/chiasekienthuc...,2025-03-09T13:00:06.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzMwMjMyNzk3OD...,1730232797872036,"[{'mediaset_token': 'pcb.1730232797872036', 'u...",NaN,NaN,NaN,NaN,NaN,"[Tech_Learning, DA]","[[Biểu đồ đã lừa bạn như thế nào?], Cherry Pic..."
3,https://www.facebook.com/groups/chiasekienthuc...,2025-02-27T11:39:11.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,**[Biểu đồ đã lừa bạn như thế nào?]**\n\n**Che...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzIyMTc5Mzc1Mz...,1722179375344045,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN,"[Tech_Learning, DA]","[[Biểu đồ đã lừa bạn như thế nào?], Cherry Pic..."
4,https://www.facebook.com/groups/chiasekienthuc...,2025-01-28T17:05:04.000Z,{'id': 'pfbid036Xm7ZRiE8DJs7WcTayLke3EFkDwC5jC...,Chúc các thành viên nhóm Chia sẻ kiến thức côn...,UzpfSTEwMDA3NzYwMTk2OTU0NjpWSzoxNzAxNjc5MjY0MD...,1701679264060723,[{'thumbnail': 'https://scontent.fapq1-1.fna.f...,NaN,NaN,NaN,NaN,NaN,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,https://www.facebook.com/groups/chiasekienthuc...,2021-12-30T12:59:32.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",# **6 điều cần biết khi bắt đầu theo đuổi Digi...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5OTg1MjI0NTc3MD...,998522457709744,[{'thumbnail': 'https://scontent.fsgn8-2.fna.f...,NaN,NaN,NaN,NaN,NaN,"[ETCLUB, UEH, DigitalMarketing]",[6 điều cần biết khi bắt đầu theo đuổi Digital...
152,https://www.facebook.com/groups/chiasekienthuc...,2021-12-27T13:07:37.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",**[GÓC REVIEW] CÁCH MÌNH ĐẠT A+ MÔN NGUYÊN LÝ ...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5OTY3MzIzOTEyMj...,996732391222084,[{'thumbnail': 'https://scontent.fsgn8-1.fna.f...,NaN,NaN,https://bit.ly/HoiDapCungET,NaN,NaN,"[ETCLUB, REVIEW, NLKT]",[[GÓC REVIEW] CÁCH MÌNH ĐẠT A+ MÔN NGUYÊN LÝ K...
154,https://www.facebook.com/groups/chiasekienthuc...,2021-12-23T12:57:13.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",🏵️ Tổ chức chương trình Hỏi Đáp cùng ET - Giải...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5OTQwNzgwMDE0OD...,994078001487523,[{'thumbnail': 'https://scontent.fmcz7-2.fna.f...,NaN,[{'url': 'https://l.facebook.com/l.php?u=https...,https://bit.ly/HoiDapCungET,NaN,NaN,[],[]
161,https://www.facebook.com/groups/chiasekienthuc...,2021-10-23T05:49:12.000Z,"{'id': '100069319561563', 'name': 'Câu lạc bộ ...",Nhóm được thành lập bởi Câu lạc bộ Công nghệ K...,UzpfSTEwMDA2OTMxOTU2MTU2MzpWSzo5NTYyMDIyMjE5ND...,956202221941768,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
